In [1]:
!pip install -q langchain groq llama-index chromadb sentence-transformers pypdf PyPDF2 langchain-community jedi pdf2image matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 112.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.1/282.1 kB 22.3 MB/s eta 0

In [168]:
!pip install -q sentence-transformers scikit-learn


In [169]:
import os
os.environ["groq_api"] = "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"


In [170]:
from google.colab import files
uploaded = files.upload()



Saving ind_geo.pdf to ind_geo (6).pdf


In [171]:
doc_path = "ind_geo.pdf"
reader = PdfReader(doc_path)
raw_text = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        raw_text += text + "\n"

In [172]:
# 2. Import libraries
import os
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
import shutil
import ipywidgets as widgets
from IPython.display import display, Image, Markdown
from pdf2image import convert_from_path
import matplotlib.pyplot as plt


In [173]:

# 5. Embeddings and vector DB
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector_store = Chroma.from_documents(docs, embedding=embedding_model, persist_directory="./groq_doc_chatbot/chroma_index")
vector_store.persist()

In [174]:
from pdf2image import convert_from_path
import os

doc_path = "ind_geo.pdf"  # Your actual file
os.makedirs("groq_doc_chatbot/images", exist_ok=True)

pages = convert_from_path(doc_path, first_page=1, last_page=200)
image_paths = []

for i, page in enumerate(pages):
    path = f"groq_doc_chatbot/images/page_{i+1}.png"
    page.save(path, "PNG")
    image_paths.append(path)

print("✅ Converted pages to images.")



✅ Converted pages to images.


In [175]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from PyPDF2 import PdfReader

reader = PdfReader("ind_geo.pdf")
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=200)

docs = []
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        splits = splitter.create_documents([text])
        for s in splits:
            s.metadata["page"] = i + 1  # 🧠 Add page number
            docs.append(s)

print(f"✅ Chunks created with page metadata. Total: {len(docs)}")


✅ Chunks created with page metadata. Total: 779


In [176]:
# 7. Groq LLM
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(
    model="llama3-8b-8192",
    temperature=0,
    openai_api_key=os.environ["groq_api"],
    openai_api_base="https://api.groq.com/openai/v1"
)

In [155]:
# 8. Memory with explicit keys
memory = ConversationBufferMemory(
    memory_key="chat_history",
    input_key="question",
    output_key="answer",
    return_messages=True
)

In [177]:

# 9. Chain
chat_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vector_store.as_retriever(search_kwargs={"k": 2}),
    memory=memory,
    return_source_documents=True,
    output_key="answer"
)


In [178]:
# 10. Widgets
query_box = widgets.Text(placeholder="Ask your question...", description="❓")
mode = widgets.Dropdown(options=["Normal", "Explain Like I'm 5", "Summarize"], value="Normal", description="🎯 Mode:")
tone = widgets.Dropdown(options=["Formal", "Casual", "Sarcastic"], value="Formal", description="🎭 Tone:")
persona = widgets.Dropdown(options=["None", "Legal Expert", "Sci-Fi Nerd", "Shiva Bhakta"], value="None", description="🧠 Persona:")
ask_button = widgets.Button(description="🚀 Ask")
output = widgets.Output()

In [179]:
# 11. Tone + persona + query injection
def apply_tone_and_persona(query, tone_val, persona_val):
    if tone_val == "Casual":
        query = f"Talk casually: {query}"
    elif tone_val == "Sarcastic":
        query = f"Respond sarcastically: {query}"

    if persona_val != "None":
        query = f"Act as a {persona_val}. {query}"
    return query

In [180]:
from IPython.display import Image, Markdown, display
import os
import time
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import numpy as np

similarity_model = SentenceTransformer("all-MiniLM-L6-v2")

def on_ask_click(b):
    with output:
        output.clear_output()
        query = query_box.value.strip()[:1000]  # ✅ Limit to 1000 characters

        if not query:
            print("⚠️ Please enter a question.")
            return

        # ✅ Mode prompt modification (applies to ALL modes)
        modified_query = query
        if mode.value == "Explain Like I'm 5":
            modified_query = "Explain like I’m five: " + query
        elif mode.value == "Summarize":
            modified_query = "Summarize: " + query

        # ✅ Apply tone and persona
        final_query = apply_tone_and_persona(modified_query, tone.value, persona.value)

        try:
            start_time = time.time()
            result = chat_chain.invoke({"question": final_query})
            end_time = time.time()

            answer = result["answer"]
            display(Markdown(f"### 🤖 **Answer:**\n{answer}"))

            token_len = len(answer.split())
            confidence = "High" if token_len > 50 else ("Medium" if token_len > 20 else "Low")
            response_time = round(end_time - start_time, 2)

            # Semantic similarity
            top_source = result["source_documents"][0].page_content[:1000]
            embeddings = similarity_model.encode([query, top_source])
            similarity_score = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]

            print(f"\n📶 Confidence: {confidence} | 🧠 Token Count: {token_len} | ⏱ Time: {response_time}s | 📏 Relevance: {round(similarity_score*100, 2)}%")

            # Source Preview
            print("\n📚 Source Preview:")
            for i, doc in enumerate(result["source_documents"][:2], 1):
                print(f"--- Source {i} ---\n{doc.page_content[:300].strip()}...\n")

            # ✅ Visuals for ALL queries (not just one mode)
            print("\n🖼 Relevant Visuals from the Source Pages:")
            image_displayed = 0
            max_images = 2
            shown_pages = set()

            for doc in result["source_documents"]:
                page_num = doc.metadata.get("page", None)
                if not page_num or page_num in shown_pages:
                    continue

                img_path = f"groq_doc_chatbot/images/page_{page_num}.png"
                if os.path.exists(img_path):
                    print(f"📄 Page {page_num}")
                    display(Image(img_path))
                    image_displayed += 1
                    shown_pages.add(page_num)

                if image_displayed >= max_images:
                    break

            if image_displayed == 0:
                print("⚠️ No visuals available for the retrieved pages.")

        except Exception as e:
            print(f"❌ Error: {str(e)}")


In [181]:
ask_button.on_click(on_ask_click)
display(query_box, mode, tone, persona, ask_button, output)


Text(value='', description='❓', placeholder='Ask your question...')

Dropdown(description='🎯 Mode:', options=('Normal', "Explain Like I'm 5", 'Summarize'), value='Normal')

Dropdown(description='🎭 Tone:', options=('Formal', 'Casual', 'Sarcastic'), value='Formal')

Dropdown(description='🧠 Persona:', options=('None', 'Legal Expert', 'Sci-Fi Nerd', 'Shiva Bhakta'), value='Non…

Button(description='🚀 Ask', style=ButtonStyle())

Output()